# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,ushuaia,-54.8000,-68.3000,6.81,70,20,1.03,AR,1738043352
1,1,bredasdorp,-34.5322,20.0403,19.77,85,100,7.85,ZA,1738043353
2,2,crane,31.3974,-102.3501,4.66,79,73,3.78,US,1738043354
3,3,enewetak,11.3474,162.3373,27.14,78,50,7.85,MH,1738043355
4,4,blackmans bay,-43.0167,147.3167,19.07,37,8,6.91,AU,1738043357


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
# Display the map

city_data_df.hvplot.scatter(
    x='Lng', y='Lat', color='City', size=30, alpha=0.5, tiles='OSM',
     xlabel='Longitude', ylabel='Latitude', width=900, height=400
)

:Overlay
   .Tiles.I   :Tiles   [x,y]
   .Scatter.I :Scatter   [x]   (y,City)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
temp_condition = (city_data_df['Max Temp'].between(15, 27)) 
wind_condition = (city_data_df['Wind Speed']<4.5)
cloud_condition = (city_data_df['Cloudiness'] == 0)

ideal_condition_df = city_data_df[
    temp_condition & 
    wind_condition &
    cloud_condition
].copy()


# Drop any rows with null values
ideal_condition_df=ideal_condition_df.dropna()

# Display sample data
ideal_condition_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
18,18,coquimbo,-29.9533,-71.3436,16.85,88,0,2.06,CL,1738043374
46,46,pisco,-13.7000,-76.2167,22.03,73,0,2.57,PE,1738043409
47,47,boa vista,2.8197,-60.6733,25.99,73,0,3.09,BR,1738043410
84,84,chinde,26.8212,87.7095,22.13,39,0,3.18,NP,1738043460
189,189,seeb,23.6703,58.1891,23.05,33,0,1.03,OM,1738043599
278,278,crucecita,-28.9500,-58.9833,24.56,63,0,2.66,AR,1738043714
327,327,chos malal,-37.3781,-70.2709,16.95,32,0,1.01,AR,1738043781
358,358,zayed city,24.3768,54.5890,23.33,81,0,3.60,AE,1738043821
398,398,acajutla,13.5928,-89.8275,25.57,67,0,3.26,SV,1738043618
426,426,rawson,-43.3002,-65.1023,16.16,44,0,3.60,AR,1738043909


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_condition_df[[
    'City', 'Country', 'Lat', 'Lng', 'Humidity'
]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
18,coquimbo,CL,-29.9533,-71.3436,88,
46,pisco,PE,-13.7000,-76.2167,73,
47,boa vista,BR,2.8197,-60.6733,73,
84,chinde,NP,26.8212,87.7095,39,
189,seeb,OM,23.6703,58.1891,33,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000
params = {
    'categories': 'accommodation.hotel',
    'limit': 1, # only need first result
    'apiKey': geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row['Lat']
    lng = row['Lng']

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f'circle:{lng},{lat},{radius}'
    params["bias"] = f'proximity:{lng},{lat}'

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    response = requests.get(base_url, params=params)

    if response.status_code != 200:
        print(f'Error: {response.status_code}, {response.text}')
        hotel_df.loc[index, 'Hotel Name'] = 'API request failed'


    # Convert the API response to JSON format
    name_address = response.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
coquimbo - nearest hotel: Hotel Iberia
pisco - nearest hotel: La Portada
boa vista - nearest hotel: Aipana Plaza Hotel
chinde - nearest hotel: No hotel found
seeb - nearest hotel: فندق جولدن كراون السيب
crucecita - nearest hotel: No hotel found
chos malal - nearest hotel: Hospedaje Lemus
zayed city - nearest hotel: L'Arabia hotel
acajutla - nearest hotel: Hotel Lara
rawson - nearest hotel: Hotel Deportivo
constitucion - nearest hotel: Hostal Alameda
taoudenni - nearest hotel: No hotel found
morant bay - nearest hotel: Morant Villas
zouerate - nearest hotel: فندق تيرس زمور
necochea - nearest hotel: Cabañas Río Quequén


,City,Country,Lat,Lng,Humidity,Hotel Name
18,coquimbo,CL,-29.9533,-71.3436,88,Hotel Iberia
46,pisco,PE,-13.7000,-76.2167,73,La Portada
47,boa vista,BR,2.8197,-60.6733,73,Aipana Plaza Hotel
84,chinde,NP,26.8212,87.7095,39,No hotel found
189,seeb,OM,23.6703,58.1891,33,فندق جولدن كراون السيب
278,crucecita,AR,-28.9500,-58.9833,63,No hotel found
327,chos malal,AR,-37.3781,-70.2709,32,Hospedaje Lemus
358,zayed city,AE,24.3768,54.5890,81,L'Arabia hotel
398,acajutla,SV,13.5928,-89.8275,67,Hotel Lara
426,rawson,AR,-43.3002,-65.1023,44,Hotel Deportivo


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

# Configure the map plot
# Display the map

hotel_df.hvplot.scatter(
    x='Lng', y='Lat', color='City', size=30, alpha=0.5, tiles='OSM',
     xlabel='Longitude', ylabel='Latitude', width=900, height=400,
     hover_cols=['Country', 'Hotel Name']
)

:Overlay
   .Tiles.I   :Tiles   [x,y]
   .Scatter.I :Scatter   [x]   (y,City,Country,Hotel Name)